In [1]:
import numpy as np
import pandas as pd 

In [2]:
!pip install accelerate==0.29.3 huggingface-hub==0.22.2 numpy==1.26.4 rouge_score==0.1.2 sentencepiece==0.2.0 torch transformers

  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.29.3
    Uninstalling huggingface-hub-0.29.3:
      Successfully uninstalled huggingface-hub-0.29.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.4.1 requires huggingface-hub>=0.24.0, but you have huggingface-hub 0.22.2 which is incompatible.


In [3]:
!pip install evaluate

  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.22.2
    Uninstalling huggingface-hub-0.22.2:
      Successfully uninstalled huggingface-hub-0.22.2


In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /teamspace/studios/this_studio/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
import nltk
nltk.download('punkt_tab')

from nltk.tokenize import sent_tokenize

test_text = "Hello world! How are you?"
try:
    print(sent_tokenize(test_text))  # Should print: ['Hello world!', 'How are you?']
except Exception as e:
    print(f"🚨 Tokenization error: {e}")


['Hello world!', 'How are you?']


[nltk_data] Downloading package punkt_tab to
[nltk_data]     /teamspace/studios/this_studio/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [6]:
from datasets import load_dataset
dataset = load_dataset("gretelai/synthetic_text_to_sql")

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'domain', 'domain_description', 'sql_complexity', 'sql_complexity_description', 'sql_task_type', 'sql_task_type_description', 'sql_prompt', 'sql_context', 'sql', 'sql_explanation'],
        num_rows: 100000
    })
    test: Dataset({
        features: ['id', 'domain', 'domain_description', 'sql_complexity', 'sql_complexity_description', 'sql_task_type', 'sql_task_type_description', 'sql_prompt', 'sql_context', 'sql', 'sql_explanation'],
        num_rows: 5851
    })
})

In [8]:
df_dataset = dataset['train'].to_pandas()

In [9]:
df_dataset.head()

,id,domain,domain_description,sql_complexity,sql_complexity_description,sql_task_type,sql_task_type_description,sql_prompt,sql_context,sql,sql_explanation
0,5097,forestry,Comprehensive data on sustainable forest manag...,single join,"only one join (specify inner, outer, cross)",analytics and reporting,"generating reports, dashboards, and analytical...",What is the total volume of timber sold by eac...,"CREATE TABLE salesperson (salesperson_id INT, ...","SELECT salesperson_id, name, SUM(volume) as to...","Joins timber_sales and salesperson tables, gro..."
1,5098,defense industry,"Defense contract data, military equipment main...",aggregation,"aggregation functions (COUNT, SUM, AVG, MIN, M...",analytics and reporting,"generating reports, dashboards, and analytical...",List all the unique equipment types and their ...,CREATE TABLE equipment_maintenance (equipment_...,"SELECT equipment_type, SUM(maintenance_frequen...",This query groups the equipment_maintenance ta...
2,5099,marine biology,"Comprehensive data on marine species, oceanogr...",basic SQL,basic SQL with a simple select statement,analytics and reporting,"generating reports, dashboards, and analytical...",How many marine species are found in the South...,"CREATE TABLE marine_species (name VARCHAR(50),...",SELECT COUNT(*) FROM marine_species WHERE loca...,This query counts the number of marine species...
3,5100,financial services,Detailed financial data including investment s...,aggregation,"aggregation functions (COUNT, SUM, AVG, MIN, M...",analytics and reporting,"generating reports, dashboards, and analytical...",What is the total trade value and average pric...,"CREATE TABLE trade_history (id INT, trader_id ...","SELECT trader_id, stock, SUM(price * quantity)...",This query calculates the total trade value an...
4,5101,energy,Energy market data covering renewable energy s...,window functions,"window functions (e.g., ROW_NUMBER, LEAD, LAG,...",analytics and reporting,"generating reports, dashboards, and analytical...",Find the energy efficiency upgrades with the h...,"CREATE TABLE upgrades (id INT, cost FLOAT, typ...","SELECT type, cost FROM (SELECT type, cost, ROW...",The SQL query uses the ROW_NUMBER function to ...


In [10]:
dataset.save_to_disk('text_to_sql_data') # save dataset to disk

Saving the dataset (0/1 shards):   0%|          | 0/100000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5851 [00:00<?, ? examples/s]

In [11]:
df_dataset_filtered = df_dataset[['sql_prompt', 'sql_context', 'sql']]

In [12]:
prompt_total = 0
context_total = 0
sql_total = 0
for idx in df_dataset.index:
    prompt = df_dataset_filtered['sql_prompt'][idx]
    context = df_dataset_filtered['sql_context'][idx]
    sql = df_dataset_filtered['sql'][idx]

    prompt_count = len(prompt.split())
    context_count = len(context.split())
    sql_count = len(sql.split())

    prompt_total+=prompt_count
    context_total+=context_count
    sql_total+=sql_count

    if idx%20000==0:
        if idx>1:
            print("total = ",idx)
            print("prompt_total average = ",prompt_total/idx)
            print("context_total average = ",context_total/idx)
            print("input_total average = ",(prompt_total+context_total)/idx)
            print("sql_total average = ",sql_total/idx)
            print()
            print()

total =  20000
prompt_total average =  13.9115
context_total average =  32.9632
input_total average =  46.8747
sql_total average =  15.7119


total =  40000
prompt_total average =  13.902125
context_total average =  32.970325
input_total average =  46.87245
sql_total average =  15.71925


total =  60000
prompt_total average =  13.88945
context_total average =  32.95778333333333
input_total average =  46.847233333333335
sql_total average =  15.689166666666667


total =  80000
prompt_total average =  13.89385
context_total average =  32.9225
input_total average =  46.81635
sql_total average =  15.6913875




In [13]:
from transformers import AutoTokenizer, BartForConditionalGeneration
from datasets import load_dataset,load_from_disk
import torch
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize
import os

In [14]:
base_path = os.getcwd()
absolute_path = os.path.join(base_path,r'text_to_sql_data')

In [15]:
dataset = load_from_disk(absolute_path,keep_in_memory=True)

In [16]:
model_checkpoint = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [17]:
inputs = tokenizer("I loved reading the Hunger Games!")
inputs

{'input_ids': [0, 100, 2638, 2600, 5, 27689, 3100, 328, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [18]:
tokenizer.convert_ids_to_tokens(inputs.input_ids)

['<s>', 'I', 'Ġloved', 'Ġreading', 'Ġthe', 'ĠHunger', 'ĠGames', '!', '</s>']

In [19]:
max_input_length = 512
max_target_length = 128
prefix1 = "sql_prompt: "
prefix2 = " sql_context: "


def preprocess_function(examples):
    model_inputs = tokenizer([prefix1 + prompt+prefix2+context for prompt,context in zip(examples['sql_prompt'],examples['sql_context'])],
        max_length=max_input_length,
        truncation=True,
    )
    labels = tokenizer(
        examples['sql'], max_length=max_target_length, truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [20]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5851 [00:00<?, ? examples/s]

In [21]:
# # Select 40k rows for training and 2.5k for testing
# small_train_dataset = tokenized_datasets["train"].select(range(100))
# small_test_dataset = tokenized_datasets["test"].select(range(25))

# from datasets import DatasetDict
# # Create a new DatasetDict with the reduced dataset
# tokenized_datasets = DatasetDict({
#     "train": small_train_dataset,
#     "test": small_test_dataset
# })

# # Print new dataset size to confirm
# print(tokenized_datasets)

In [22]:
import evaluate

rouge_score = evaluate.load("rouge")

In [23]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
model.generation_config.max_new_tokens = 128
model.generation_config.min_new_tokens = 5
model.config.max_new_tokens = 128
model.config.min_new_tokens = 5
print(model.config)

BartConfig {
  "_name_or_path": "facebook/bart-large-cnn",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "force_bos_token_to_be_generated": true,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "L

In [24]:
print(model.generation_config)

GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "length_penalty": 2.0,
  "max_length": 142,
  "max_new_tokens": 128,
  "min_length": 56,
  "min_new_tokens": 5,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1
}



In [25]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
tokenized_datasets = tokenized_datasets.remove_columns(dataset["train"].column_names)
features = [tokenized_datasets["train"][i] for i in range(2)]
data_collator(features)

{'input_ids': tensor([[    0, 49089,  1215, 12501,  3320,    35,   653,    16,     5,   746,
          3149,     9, 20747,  1088,    30,   349,   647,  5970,     6, 24713,
            30,   647,  5970,   116, 49569,  1215, 46796,    35, 28122,  8625,
         41910,   647,  5970,    36,    29,  4575,  5970,  1215,   808, 30497,
             6,   766, 36367,     6,   976, 36367,  4397, 23395, 18854, 42068,
           647,  5970,    36,    29,  4575,  5970,  1215,   808,     6,   766,
             6,   976,    43, 27445,  8888,    36,   134,     6,   128, 10567,
         28484,  3934,   128, 11073, 41734,    36,   176,     6,   128, 35903,
          1259,  3934,   128, 10050, 47033, 28122,  8625, 41910, 20747,  1215,
            29,  4575,    36,    29,  4575,  1215,   808, 30497,     6,   647,
          5970,  1215,   808, 30497,     6,  3149, 30323,     6,  1392,  1215,
         10672,   211,  8625,  4397, 23395, 18854, 42068, 20747,  1215,    29,
          4575,    36,    29,  4575,  

In [26]:
from torch.utils.data import DataLoader

batch_size = 8
train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=batch_size,
)
eval_dataloader = DataLoader(
    tokenized_datasets["test"], collate_fn=data_collator, batch_size=batch_size
)

In [27]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

In [28]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [29]:
from transformers import get_scheduler

num_train_epochs = 2
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [30]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # ROUGE expects a newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels

In [31]:
from tqdm.auto import tqdm
import torch
import numpy as np

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for step, batch in enumerate(train_dataloader):
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
        

    # Evaluation
    model.eval()
    for step, batch in enumerate(tqdm(eval_dataloader)):
        with torch.no_grad():
            generated_tokens = accelerator.unwrap_model(model).generate(
                batch["input_ids"],
                attention_mask=batch["attention_mask"],
            )

            generated_tokens = accelerator.pad_across_processes(
                generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
            )
            labels = batch["labels"]

            # If we did not pad to max length, we need to pad the labels too
            labels = accelerator.pad_across_processes(
                batch["labels"], dim=1, pad_index=tokenizer.pad_token_id
            )

            generated_tokens = accelerator.gather(generated_tokens).cpu().numpy()
            labels = accelerator.gather(labels).cpu().numpy()

            # Replace -100 in the labels as we can't decode them
            labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
            if isinstance(generated_tokens, tuple):
                generated_tokens = generated_tokens[0]
            decoded_preds = tokenizer.batch_decode(
                generated_tokens, skip_special_tokens=True
            )
            decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

            decoded_preds, decoded_labels = postprocess_text(
                decoded_preds, decoded_labels
            )

            rouge_score.add_batch(predictions=decoded_preds, references=decoded_labels)

    # Compute metrics
    result = rouge_score.compute()
    # Extract the median ROUGE scores
    result = {key: value * 100 for key, value in result.items()}
    result = {k: round(v, 4) for k, v in result.items()}
    print(f"Epoch {epoch}:", result)
    model.train()

    # Save and upload
    output_dir = 'nl2sql_epoch'+str(epoch+1)
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)

  0%|          | 0/25000 [00:00<?, ?it/s]

  0%|          | 0/732 [00:00<?, ?it/s]

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


Epoch 0: {'rouge1': 81.1405, 'rouge2': 68.9389, 'rougeL': 78.0204, 'rougeLsum': 78.0184}


  0%|          | 0/732 [00:00<?, ?it/s]

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


Epoch 1: {'rouge1': 82.119, 'rouge2': 70.3602, 'rougeL': 79.0583, 'rougeLsum': 79.0415}
